In [3]:
import pandas as pd

annotations = pd.read_csv('annotations.csv')

display(annotations.head())

demographics =[
    "clinic_gender",
    "clinic_age",
    "clinic_ethnicity",
    "education",
    "current_smoking_status",
    "age_start_smoking_daily",
    "age_stopped_smoking",
    "years_smoked",
    "years_quit",
    "average_lifetime_cigarettesperda",
    "pack_years",
    "FEV1_abs",
    "FEV1_PP",
    "FVC_abs",
    "FVC_PP",
    "FEV1FVC",
    "Height",
    "Weight",
    "BMI",
    "GP_comment" 
]

scan_variables = [
    "Date_BaselineCT",
    "Date_BaselineReport",
    "time_to_report",
    "Reader_init",
    "Reader_init_other",
    "Prev_Scan",
    "Start_time_BaselineCT",
    "Stop_time_BaselineCT",
    "report_time",
    "CT_FU",
    "Imp1",
    "Imp2",
    "Tumour",
    "CT_T",
    "CT_N",
    "CT_M",
    "CT_stage",
    "Stage_reas",
    "Total_no_nods",
    "Brock_sc",
    "LungRADS_cat",
    "Recall_reason",
    "Emphysema",
    "Bronchiectasis",
    "Bronch_thick",
    "Muc_plug",
    "IPFlike_ILD",
    "nonIPFlike_ILD",
    "CAC",
    "Osteoporosis",
    "Cardiomegaly",
    "PE",
    "enlargedPA",
    "OA",
    "Aort_an",
    "Neck_mass",
    "Med_LN",
    "Hilar_LN",
    "Med_mass",
    "Liver_les",
    "Kid_les",
    "Adr_les",
    "Hiat_hern",
    "Pl_effusion",
    "Pericard_effusion",
    "Pl_thick_noncalc",
    "Pl_plaq_calc",
    "Bone_les",
    "Rib_frac",
    "Vert_frac",
    "Other_cardiac",
    "Other_incident" 
]

nodule_variables = [
    "_diam",
    "_vol",
    "_dens",
    "_type",
    "_type_other",
    "_lobe",
    "_loc",
    "_pos",
    "_pos_other",
    "_round",
    "_ovoid",
    "_amorph",
    "_tub",
    "_shape_other",
    "_irreg",
    "_spic",
    "_lobulated",
    "_smooth",
    "_margin_other",
    "_fat",
    "_calc",
    "_air_bronch",
    "_bubble",
    "_pl_teth",
    "_Pl_rxn",
]

nodule_all_variables = [
    "ScananonID",
    "Nod_comments"
]

,ScananonID,Date_study_entry,clinic_gender,clinic_age,clinic_ethnicity,education,current_smoking_status,age_start_smoking_daily,age_stopped_smoking,years_smoked,...,feb_Clin_T,feb_Clin_N,feb_Clin_M,feb_Clin_stage,feb_Path_T,feb_Path_N,feb_Path_M,feb_Path_PL,feb_Path_R,feb_Path_stage
0,UCLH_88113396,04apr2016,Male,67,White,1. Finished school at or before the age of fif...,Current smoker,20,NaN,47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,UCLH_50882667,05apr2016,Male,69,White,"2. Completed CSEs, O-levels or equivalent",Current smoker,17,NaN,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,UCLH_20466996,06apr2016,Male,66,White,1. Finished school at or before the age of fif...,Former smoker,16,62.0,46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,UCLH_90062266,07apr2016,Male,65,White,1. Finished school at or before the age of fif...,Current smoker,14,NaN,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,UCLH_61426138,07apr2016,Male,70,White,6. Completed a further degree e.g. masters/PhD...,Former smoker,14,60.0,46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
annotations['Total_no_nods'].value_counts().sort_index()

0.0     580
1.0     115
2.0      24
3.0       9
4.0       6
5.0       5
6.0       2
8.0       3
10.0      8
12.0      1
15.0      3
16.0      1
20.0      1
25.0      1
50.0      1
Name: Total_no_nods, dtype: int64

# Generate Dataset json for Detection


In [17]:
import pandas as pd

mhd_metadata = (
    pd.read_csv('scans/metadata.csv')
    .drop(columns=['StudyId','scan_id'])
    .assign(study_id=lambda df: df['mhd_path'].str.split('/').str[6])
    .assign(scan_id=lambda df: df.study_id)
    .filter(
        [
            'study_id',
            'scan_id',
            'x-offset',
            'y-offset',
            'z-offset',
            'x-spacing',
            'y-spacing',
            'z-spacing', 
            'x-pixels', 
            'y-pixels', 
            'slices'
        ]
    )
)
# mhd_metadata['mhd_path'].str.split('/', expand=True)

mhd_metadata

,study_id,scan_id,x-offset,y-offset,z-offset,x-spacing,y-spacing,z-spacing,x-pixels,y-pixels,slices
0,UCLH_00134949,UCLH_00134949,-160.0000,-160.0000,-1664.0000,0.625,0.625,0.800000,512.0,512.0,326.0
1,UCLH_14272937,UCLH_14272937,-160.0000,-160.0000,-1146.5990,0.625,0.625,0.799997,512.0,512.0,388.0
2,UCLH_22120065,UCLH_22120065,-203.6876,-194.8986,1303.6000,0.827,0.827,0.800000,512.0,512.0,424.0
3,UCLH_12346090,UCLH_12346090,-199.6093,-199.6093,1518.1000,0.781,0.781,0.800000,512.0,512.0,364.0
4,UCLH_03393370,UCLH_03393370,-148.0865,-179.3365,-579.6000,0.702,0.702,0.800000,512.0,512.0,383.0
...,...,...,...,...,...,...,...,...,...,...,...
700,UCLH_98003335,UCLH_98003335,-178.5568,-178.5568,1631.6000,0.698,0.698,0.800000,512.0,512.0,409.0
701,UCLH_98028911,UCLH_98028911,-137.9776,-145.7901,1470.6000,0.607,0.607,0.800000,512.0,512.0,379.0
702,UCLH_98051606,UCLH_98051606,-197.6560,-195.3125,-989.5999,0.763,0.763,0.800000,512.0,512.0,313.0
703,UCLH_98143067,UCLH_98143067,-159.6875,-159.6875,1670.7000,0.625,0.625,0.800000,512.0,512.0,382.0
